<h1>Basic Strategy Setup</h1>
<h3>Investment Universe: US Equities Listed on NYSE or NASDAQ with daily return less than 5%</h3>
<h3>Initial Condition: 1 day return <= -5%</h3>
<h3>Holding Period: Enter long position at market close, exit at next day market open</h3>

In [1]:
import pandas as pd
from datetime import datetime
import requests
import numpy as np
from polygon import RESTClient
from tqdm import tqdm
import warnings
import pytz
from concurrent.futures import ThreadPoolExecutor, as_completed
import concurrent.futures
import requests
from io import StringIO

warnings.simplefilter(action='ignore', category=FutureWarning)

api_key = 'AKoq4vo8GVYNnuWg9Fw9xv0H93HFXv_m'
client = RESTClient(api_key)

<h1>(1): Collect dates and corresponding active tickers</h1>
<h3>Date range: Jan 2022 - Dec 2023</h3>
<h3>Ticker Types: US Equities Listed on NYSE or NASDAQ (no OTC tickers)</h3>

In [41]:
#Date Retrieval
all_dates = pd.date_range(start='2020-01-01', end='2024-01-01').strftime('%Y-%m-%d').tolist() #Generate list of all dates in range
valid_dates = [] 
for date in tqdm(all_dates, desc="Validating Dates"): #iterate through all dates, saving any that SPY traded on
    response = requests.get(f'https://api.polygon.io/v2/aggs/ticker/SPY/range/1/day/{date}/{date}?adjusted=true&sort=asc&apiKey={api_key}')
    if response.json()['queryCount'] == 1:
        valid_dates.append(date)
print(f'Valid Dates Count:{len(valid_dates)}, All Dates Count:{len(all_dates)}')

Validating Dates: 100%|██████████| 1462/1462 [03:25<00:00,  7.13it/s]

Valid Dates Count:1006, All Dates Count:1462


In [35]:
#Select Date Range
print(f'{valid_dates[505:][0]} - {valid_dates[505:][-1]}')

2022-01-03 - 2023-12-29


In [36]:
#ticker retrieval function: pulls active ticker list
def get_tickers(date):
    with requests.Session() as s: 
        response = requests.get(f'https://www.alphavantage.co/query?function=LISTING_STATUS&date={date}&state=active&apikey=UHFFT76TTRPC75V6')
        decoded_content = response.content.decode('utf-8')
        df = pd.read_csv(StringIO(decoded_content))
        tlist = list(df['symbol'])
        tlist = [ticker for ticker in tlist if isinstance(ticker, str) and '.' not in ticker and '-' not in ticker and '/' not in ticker]
        tlist  = [item for item in tlist if len(item) <= 4]
        return tlist

ticker_date_dict = {date: get_tickers(date) for date in tqdm(valid_dates[505:], desc="Fetching tickers")}
all_unique_tickers = list({ticker for tickers in ticker_date_dict.values() for ticker in tickers}) #get list of all active tickers within date range

Fetching tickers: 100%|██████████| 501/501 [05:54<00:00,  1.41it/s]


<h1>(2): Scan for daily return less than -5%</h1>

In [12]:
#collect 1d aggregate data, filter by 1d return <=-5%
def daily_aggs(ticker):
    try:
        #Get daily aggregates
        data = client.get_aggs(ticker,
                                    multiplier=1,
                                    timespan='day',
                                    from_='2021-01-01',
                                    to='2023-01-01')
        df = pd.DataFrame(data)
        #Check for data presence, filter OTC tickers
        if df.empty or df[df['otc'] != True].empty:
            return None
        #add identifiers
        df['ticker'] = ticker
        df['date'] = df['timestamp'].apply(est)
        #get 1d return change %, filter by <=-5%, volume >= 50000
        df['1d_return'] = round(df['close'].pct_change(),4)
        df = df[df['1d_return'] <= -0.05]
        df = df[df['volume'] >= 50000]
        #Check for data presence post-filter, proceed if possible
        if not df.empty:
            #format return row(s)
            df = df.drop(columns=['open','high','low','close','vwap','otc','transactions'])
            #dict->flatten
            records_list = df.to_dict(orient='records')
            single_dict = records_list if records_list else {}
            return single_dict
    except Exception as e:
        print(f"An error occurred for {ticker}: {e}")
        return None

with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
    results = list(tqdm(executor.map(daily_aggs, all_unique_tickers), total=len(all_unique_tickers)))

results = [item for sublist in results if sublist is not None for item in sublist]
basedf = pd.DataFrame(results)
basedf.info()

100%|██████████| 10745/10745 [02:51<00:00, 62.54it/s]


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133713 entries, 0 to 133712
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   volume     133713 non-null  float64
 1   timestamp  133713 non-null  int64  
 2   1d_return  133713 non-null  float64
 3   Ticker     133713 non-null  object 
dtypes: float64(2), int64(1), object(1)
memory usage: 4.1+ MB


<h1>(3): Collect Feature and Return Outcome Data</h1>

In [13]:
#Import date processing functions
def unix(date_str,hour,minute):
    date = datetime.strptime(date_str, '%Y-%m-%d').date()
    # Define Eastern Standard Time (EST) timezone
    est_tz = pytz.timezone('America/New_York')
    # Define the time
    est_time = datetime(date.year, date.month, date.day, hour, minute, 0)
    # Localize the datetime object to EST timezone
    est_dt = est_tz.localize(est_time, is_dst=None)
    # Convert the datetime object to UTC timezone
    utc_dt = est_dt.astimezone(pytz.utc)
    # Calculate the Unix timestamp in milliseconds
    unix_timestamp_ms = int((utc_dt - datetime(1970, 1, 1, tzinfo=pytz.utc)).total_seconds() * 1000)
    return unix_timestamp_ms

def est(unix_ms_timestamp):
    # Convert Unix timestamp in milliseconds to seconds
    unix_seconds = unix_ms_timestamp / 1000.0
    # Create a datetime object from the Unix timestamp
    utc_time = datetime.utcfromtimestamp(unix_seconds)
    # Define the UTC and EST timezones
    utc_zone = pytz.utc
    est_zone = pytz.timezone('US/Eastern')
    # Localize the UTC datetime object to UTC timezone
    utc_time = utc_zone.localize(utc_time)
    # Convert the UTC time to EST
    est_time = utc_time.astimezone(est_zone)
    est_time = est_time.replace(tzinfo=None)
    return est_time.strftime('%Y-%m-%d')

In [39]:
#Daily aggregate-based variable function
def daily_vars(ticker,op_date):
    try:
        ret_dict = {}
        #get dates
        start_date = valid_dates[valid_dates.index(op_date) - 20] #start 20 days ago
        end_date = valid_dates[valid_dates.index(op_date) + 1] #end 1 day ahead

        data = client.get_aggs(ticker,
                                    multiplier=1,
                                    timespan='day',
                                    from_=start_date,
                                    to=end_date)
        df = pd.DataFrame(data)
        df['Date'] = df['timestamp'].apply(est)
        #check for missing data
        if (df[df['Date'] == op_date].empty or df[df['Date'] == end_date].empty or df[df['Date'] == valid_dates[valid_dates.index(op_date) - 1]].empty):
            return {}
        #create features - close return periods
        ret_dict['op_close'] = df[df['Date'] == op_date]['close'].values[0]
        ret_dict['return_target'] = round((df[df['Date'] == end_date]['open'].values[0]/ret_dict['op_close'])-1,4)
        ret_dict['1d_intraday_return'] = round((ret_dict['op_close']/df[df['Date'] == op_date]['open'].values[0])-1,4) if not df[df['Date'] == op_date].empty else np.nan
        ret_dict['1d_afterhours_return']  = round((df[df['Date'] == op_date]['open'].values[0] / df[df['Date'] == valid_dates[valid_dates.index(op_date) - 1]]['close'].values[0]) - 1, 4)
        #create features - far return periods
        for prevday in [2,3,4,5,6,7,8,9,10,15,20]:
            if not df[df['Date'] == valid_dates[valid_dates.index(op_date) - prevday]].empty:
                ret_dict[f'{prevday}d_return'] = round((ret_dict['op_close']/df[df['Date'] == valid_dates[valid_dates.index(op_date) - prevday]]['close'].values[0])-1,4)
            else:
                ret_dict[f'{prevday}d_return'] = np.nan 
        return ret_dict
    except Exception as e:
        print(f"An exception error occurred for {ticker} on {date}: {e}")
        return {}

daily_vars('AAPL','2023-02-03')

{'op_close': 154.5,
 'return_target': -0.0125,
 '1d_intraday_return': 0.0437,
 '1d_afterhours_return': -0.0185,
 '2d_return': 0.0624,
 '3d_return': 0.0708,
 '4d_return': 0.0804,
 '5d_return': 0.0587,
 '6d_return': 0.0732,
 '7d_return': 0.0891,
 '8d_return': 0.084,
 '9d_return': 0.0949,
 '10d_return': 0.1206,
 '15d_return': 0.1581,
 '20d_return': 0.2358}

In [2]:
#Minute aggregate-based variable function
def minute_vars(ticker,date_string,op_close):
    try:
        if np.isnan(op_close):
            return {}
        
        min_dict = {}
        #reformat timestamps
        start_ts= unix(date_string,14,0)
        end_ts = unix(date_string,15,29)
        #call API
        data = client.get_aggs(ticker,
                                    multiplier=1,
                                    timespan='minute',
                                    from_=start_ts,
                                    to=end_ts)
        df = pd.DataFrame(data)
        if df.empty:
            return {}
        #2h return
        twodf = df[(df['timestamp'] >= unix(date_string,14,0)) & (df['timestamp'] <= unix(date_string,14,29))]
        if not twodf.empty:
            min_dict['2h_return'] = round((op_close/twodf['open'].iloc[0])-1,4)
        else:
            min_dict['2h_return'] = np.nan
        
        #1h return
        threedf = df[(df['timestamp'] >= unix(date_string,15,0)) & (df['timestamp'] <= unix(date_string,15,29))]
        if not threedf.empty:
            min_dict['1h_return'] = round((op_close/threedf['open'].iloc[0])-1,4)
        else:
            min_dict['1h_return'] = np.nan

        return min_dict
    except Exception as e:
        print(f"An exception error occurred for {ticker} on {date}: {e}")
        return {}

In [17]:
with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
    daily_results = list(tqdm(executor.map(daily_vars, basedf['ticker'], basedf['timestamp']), total=len(basedf)))
daily_df = pd.DataFrame(daily_results)

with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
    minute_results = list(tqdm(executor.map(minute_vars, basedf['ticker'], basedf['date'], daily_df['op_close']), total=len(basedf)))
minute_df = pd.DataFrame(minute_results)

finaldf = pd.concat([basedf, daily_df, minute_df], axis=1)

100%|██████████| 133713/133713 [17:35<00:00, 126.67it/s]


In [25]:
#Get spy data
spydata = client.get_aggs('SPY',
                        multiplier=1,
                        timespan='day',
                        from_='2021-01-01',
                        to='2024-01-01')
spydf = pd.DataFrame(spydata)
spydf['date'] = spydf['timestamp'].apply(est)
spydf['spy_1d_return'] = round(spydf['close'].pct_change(),4)
finaldf = finaldf.merge(spydf[['date', 'spy_1d_return']], on='date', how='left')

In [27]:
finaldf['date'] = finaldf['timestamp'].apply(est)
finaldf = finaldf.merge(spydf[['date', 'spy_1d_return']], on='date', how='left')
finaldf.to_csv('main_raw.csv')

In [31]:
finaldf.to_csv('main_raw.csv')